In [ ]:
import numpy as np
import tensorflow as tf
# import pyttsx3 # Bỏ qua TTS theo yêu cầu trước
import re
import pandas as pd # Thư viện để làm việc với CSV
from collections import deque # Dùng cho hàng đợi câu hỏi

# --- Hằng số và Cấu hình ---
DISEASES_CSV = 'diseases.csv'
SYMPTOMS_CSV = 'symptoms.csv'
DISEASE_SYMPTOMS_MATRIX_CSV = 'disease_symptoms_matrix.csv'

# --- Tải và Chuẩn bị Dữ liệu ---
def load_data():
    diseases_df = pd.read_csv(DISEASES_CSV)
    symptoms_df = pd.read_csv(SYMPTOMS_CSV)
    matrix_df = pd.read_csv(DISEASE_SYMPTOMS_MATRIX_CSV)

    TEN_BENH_LIST = diseases_df['TenBenh'].tolist()
    
    # Lấy danh sách các triệu chứng chính (sẽ là feature cho model)
    TRIEU_CHUNG_CHINH_DF = symptoms_df[symptoms_df['LaTrieuChungChinh'] == True]
    MA_TRIEU_CHUNG_CHINH_LIST = TRIEU_CHUNG_CHINH_DF['MaTrieuChung'].tolist()
    TEN_TRIEU_CHUNG_CHINH_LIST = TRIEU_CHUNG_CHINH_DF['TenTrieuChung'].tolist() # Tên để hiển thị

    # Tạo X_train và y_train từ matrix_df
    # Đảm bảo các cột triệu chứng trong matrix_df khớp và đúng thứ tự với MA_TRIEU_CHUNG_CHINH_LIST
    # Nếu matrix_df có các cột không phải là mã triệu chứng (ví dụ: TenBenh), chúng ta cần loại bỏ
    feature_columns = [col for col in matrix_df.columns if col in MA_TRIEU_CHUNG_CHINH_LIST]
    
    # Sắp xếp lại các cột trong matrix_df theo đúng thứ tự của MA_TRIEU_CHUNG_CHINH_LIST nếu cần
    # Điều này quan trọng để đảm bảo tính nhất quán
    X_train_df = matrix_df[MA_TRIEU_CHUNG_CHINH_LIST] 
    X_train = X_train_df.values.astype(np.float32)

    # Chuyển đổi TenBenh thành dạng số (label encoding) rồi one-hot encoding
    disease_to_id = {name: i for i, name in enumerate(TEN_BENH_LIST)}
    y_labels = matrix_df['TenBenh'].map(disease_to_id).values
    y_train = tf.keras.utils.to_categorical(y_labels, num_classes=len(TEN_BENH_LIST))

    # Tạo dictionary cho từ khóa trích xuất
    TU_KHOA_TRIEU_CHUNG_MAP = {}
    for _, row in TRIEU_CHUNG_CHINH_DF.iterrows():
        keywords_str = row['TuKhoaChinh']
        if pd.notna(keywords_str): # Kiểm tra nếu không phải NaN
            TU_KHOA_TRIEU_CHUNG_MAP[row['MaTrieuChung']] = [r'\b' + kw.strip() + r'\b' for kw in keywords_str.split(',')]
        else:
            TU_KHOA_TRIEU_CHUNG_MAP[row['MaTrieuChung']] = []


    # Tạo dictionary các câu hỏi làm rõ
    # Key: MaTrieuChungChinh, Value: list các MaTrieuChungCon
    CAU_HOI_LAM_RO_MAP = {}
    for _, row in symptoms_df[symptoms_df['LaTrieuChungChinh'] == False].iterrows():
        parent_symptom_code = row['TrieuChungCha']
        if pd.notna(parent_symptom_code):
            if parent_symptom_code not in CAU_HOI_LAM_RO_MAP:
                CAU_HOI_LAM_RO_MAP[parent_symptom_code] = []
            CAU_HOI_LAM_RO_MAP[parent_symptom_code].append(row['MaTrieuChung'])
            
    # Map MaTrieuChung với CauHoi
    MA_TO_CAUHOI_MAP = symptoms_df.set_index('MaTrieuChung')['CauHoi'].to_dict()
    MA_TO_TEN_MAP = symptoms_df.set_index('MaTrieuChung')['TenTrieuChung'].to_dict()


    return (X_train, y_train, TEN_BENH_LIST, MA_TRIEU_CHUNG_CHINH_LIST, TEN_TRIEU_CHUNG_CHINH_LIST,
            symptoms_df, TU_KHOA_TRIEU_CHUNG_MAP, CAU_HOI_LAM_RO_MAP, MA_TO_CAUHOI_MAP, MA_TO_TEN_MAP)


# --- Xây dựng và huấn luyện mô hình ---
def build_and_train_model(X_train, y_train, num_features, num_classes):
    inputs = tf.keras.Input(shape=(num_features,))
    x = tf.keras.layers.Dense(32, activation='relu')(inputs) # Tăng số neuron một chút
    x = tf.keras.layers.Dropout(0.5)(x, training=True)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x, training=True)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Kiểm tra nếu X_train và y_train không rỗng
    if X_train.shape[0] == 0 or y_train.shape[0] == 0:
        print("LỖI: Dữ liệu huấn luyện X_train hoặc y_train rỗng. Vui lòng kiểm tra file CSV.")
        return None
    if X_train.shape[0] != y_train.shape[0]:
        print(f"LỖI: Số lượng mẫu trong X_train ({X_train.shape[0]}) và y_train ({y_train.shape[0]}) không khớp.")
        return None

    print(f"Huấn luyện mô hình với {X_train.shape[0]} mẫu, {X_train.shape[1]} đặc trưng, và {y_train.shape[1]} lớp bệnh.")
    model.fit(X_train, y_train, epochs=150, verbose=0, batch_size=min(32, X_train.shape[0])) # Thêm batch_size
    return model

# --- Chức năng Text-to-Speech (TTS) - (Có thể bật lại nếu cần) ---
# def speak_vietnamese(text): ... (như cũ)

# --- Dự đoán với độ không chắc chắn (Monte Carlo Dropout) ---
def predict_with_uncertainty(model, x_input, n_iter=100):
    if x_input.ndim == 1: # Nếu chỉ là 1 mẫu, reshape lại
        x_input = np.expand_dims(x_input, axis=0)
    preds_list = [model(x_input, training=True).numpy() for _ in range(n_iter)]
    preds = np.array(preds_list)
    mean_probs = preds.mean(axis=0)
    std_devs = preds.std(axis=0)
    return mean_probs, std_devs

# --- Xử lý đầu vào ngôn ngữ tự nhiên đơn giản cho câu hỏi Có/Không ---
def interpret_yes_no_vietnamese(answer_text):
    answer = answer_text.strip().lower()
    positive_responses = ["có", "c", "yes", "y", "đúng", "phải", "roi", "co", " bị"]
    # Thêm một số từ phủ định cơ bản
    negative_responses = ["không", "k", "no", "n", "sai", "chưa", "khong", "ko", "đéo"] 
    
    for neg_word in negative_responses:
        if neg_word in answer:
            # Xử lý trường hợp "không có" vs "có"
            is_truly_negative = True
            for pos_word in positive_responses:
                if pos_word in answer and answer.find(pos_word) < answer.find(neg_word): # "có ... không"
                    # Đây có thể là một câu hỏi, hoặc "có nhưng không nhiều", cần xử lý tinh vi hơn
                    # Tạm thời, nếu "có" xuất hiện trước "không" thì vẫn coi là có
                    # is_truly_negative = False # Cân nhắc lại logic này
                    pass
            if is_truly_negative:
                return 0 # Nếu có từ phủ định và không bị ghi đè bởi từ khẳng định đứng trước

    for pos_word in positive_responses:
        if pos_word in answer:
            return 1
            
    # Nếu không rõ ràng, hỏi lại hoặc mặc định là không
    # print("[Bot]: Xin lỗi, tôi chưa hiểu rõ câu trả lời của bạn. Bạn có thể nói rõ hơn là 'Có' hay 'Không' được không?")
    return 0 # Mặc định là không nếu không rõ

# --- Trích xuất triệu chứng cơ bản từ văn bản tự do ---
def extract_initial_symptoms_from_text(text_input, tu_khoa_map, ma_trieu_chung_chinh_list):
    detected_symptoms_values = {ma_tc: 0 for ma_tc in ma_trieu_chung_chinh_list}
    text_lower = text_input.lower()

    for ma_tc, keyword_patterns in tu_khoa_map.items():
        if ma_tc in detected_symptoms_values: # Chỉ xét các triệu chứng chính
            for pattern in keyword_patterns:
                try:
                    if re.search(pattern, text_lower):
                        detected_symptoms_values[ma_tc] = 1
                        break
                except Exception as e:
                    print(f"Lỗi regex với pattern '{pattern}': {e}")
                    continue
    return detected_symptoms_values

# --- Chạy Chatbot ---
def run_intelligent_chatbot(model, TEN_BENH_LIST, MA_TRIEU_CHUNG_CHINH_LIST, TEN_TRIEU_CHUNG_CHINH_LIST,
                            symptoms_df, tu_khoa_map, cau_hoi_lam_ro_map, ma_to_cauhoi_map, ma_to_ten_map):
    if model is None:
        print("Không thể khởi chạy chatbot vì mô hình chưa được huấn luyện.")
        return

    print("\nXin chào! Tôi là robot trợ lý sức khỏe ảo. Hãy mô tả các triệu chứng chính bạn đang gặp phải.")
    user_initial_description = input("Bạn: ")

    # 1. Trích xuất triệu chứng ban đầu
    # Đây sẽ là dictionary {MaTrieuChungChinh: 0 hoặc 1}
    current_symptom_values_map = extract_initial_symptoms_from_text(user_initial_description, tu_khoa_map, MA_TRIEU_CHUNG_CHINH_LIST)
    
    print("\n[Bot]: Dựa trên mô tả của bạn, tôi ghi nhận các triệu chứng sau:")
    has_initial_symptoms = False
    for ma_tc, present in current_symptom_values_map.items():
        if present:
            print(f"- {ma_to_ten_map.get(ma_tc, ma_tc)}") # Lấy tên triệu chứng để hiển thị
            has_initial_symptoms = True
    if not has_initial_symptoms:
        print("(Không phát hiện triệu chứng nào từ mô tả ban đầu)")

    # 2. Xây dựng hàng đợi câu hỏi thông minh
    # Ưu tiên câu hỏi làm rõ cho các triệu chứng đã báo cáo, sau đó là các triệu chứng chính khác
    question_queue = deque()
    asked_questions = set() # Để tránh hỏi lặp lại

    # Thêm câu hỏi làm rõ cho các triệu chứng đã được phát hiện
    for ma_tc_chinh, present in current_symptom_values_map.items():
        if present and ma_tc_chinh in cau_hoi_lam_ro_map:
            for ma_tc_con in cau_hoi_lam_ro_map[ma_tc_chinh]:
                if ma_tc_con not in asked_questions:
                    question_queue.append(ma_tc_con)
                    asked_questions.add(ma_tc_con)
    
    # Thêm các câu hỏi về triệu chứng chính chưa được đề cập hoặc chưa có câu trả lời
    for i, ma_tc_chinh in enumerate(MA_TRIEU_CHUNG_CHINH_LIST):
        if current_symptom_values_map.get(ma_tc_chinh, 0) == 0: # Nếu chưa được phát hiện hoặc chưa được hỏi
             if ma_tc_chinh not in asked_questions:
                question_queue.append(ma_tc_chinh) # Thêm mã triệu chứng chính vào hàng đợi
                asked_questions.add(ma_tc_chinh)

    print("\n[Bot]: Để hiểu rõ hơn, tôi xin hỏi thêm một số câu:")
    
    # Lưu trữ câu trả lời cho các câu hỏi làm rõ (không trực tiếp vào model features)
    detailed_answers = {}

    while question_queue:
        ma_tc_to_ask = question_queue.popleft()
        question_text = ma_to_cauhoi_map.get(ma_tc_to_ask, f"Bạn có bị {ma_to_ten_map.get(ma_tc_to_ask, ma_tc_to_ask).lower()} không?")
        
        # Kiểm tra nếu triệu chứng chính đã được trả lời rồi thì không hỏi lại
        # (Ví dụ: nếu TC001_1 (con của TC001) được hỏi, và TC001 chưa được xác nhận là 1, thì vẫn hỏi TC001)
        # Logic này cần tinh chỉnh thêm nếu các câu hỏi con phức tạp
        
        ans_text = input(f"[Bot]: {question_text} ")
        
        # Kiểm tra xem đây là câu hỏi cho triệu chứng chính hay câu hỏi làm rõ
        is_main_symptom_q = symptoms_df.loc[symptoms_df['MaTrieuChung'] == ma_tc_to_ask, 'LaTrieuChungChinh'].iloc[0]

        if is_main_symptom_q:
            # Đây là câu hỏi về một triệu chứng chính
            answer_value = interpret_yes_no_vietnamese(ans_text)
            current_symptom_values_map[ma_tc_to_ask] = answer_value
            # Nếu người dùng trả lời "Có" cho một triệu chứng chính, thêm các câu hỏi con của nó vào hàng đợi (nếu có và chưa hỏi)
            if answer_value == 1 and ma_tc_to_ask in cau_hoi_lam_ro_map:
                for ma_tc_con in cau_hoi_lam_ro_map[ma_tc_to_ask]:
                    if ma_tc_con not in asked_questions:
                        question_queue.appendleft(ma_tc_con) # Ưu tiên hỏi câu hỏi con ngay
                        asked_questions.add(ma_tc_con)
        else:
            # Đây là câu hỏi làm rõ, lưu câu trả lời text (có thể xử lý sau này)
            detailed_answers[ma_tc_to_ask] = ans_text.strip()


    print("\n[Bot]: Cảm ơn bạn đã cung cấp thông tin. Đang phân tích...")
    
    # Chuẩn bị input_vector cho model từ current_symptom_values_map
    # Đảm bảo thứ tự của input_vector khớp với MA_TRIEU_CHUNG_CHINH_LIST
    input_vector = np.array([current_symptom_values_map.get(ma_tc, 0) for ma_tc in MA_TRIEU_CHUNG_CHINH_LIST], dtype=np.float32)

    # 3. Dự đoán
    mean_probabilities, std_dev_probabilities = predict_with_uncertainty(model, input_vector)

    most_likely_index = np.argmax(mean_probabilities[0])
    diagnosis = TEN_BENH_LIST[most_likely_index]
    
    print("\n--- Chẩn đoán với Xác suất và Độ không chắc chắn ---")
    for i, benh_name in enumerate(TEN_BENH_LIST):
        print(f"{benh_name}: Xác suất = {mean_probabilities[0][i]:.3f}, Độ không chắc chắn (StdDev) = {std_dev_probabilities[0][i]:.3f}")

    print(f"\nChẩn đoán sơ bộ: {diagnosis} (Độ không chắc chắn cho chẩn đoán này: ±{std_dev_probabilities[0][most_likely_index]:.3f})")
    
    # (Tùy chọn) Hiển thị các khuyến nghị về xét nghiệm và thuốc (cần map từ diseases.csv)
    # ... (Thêm logic lấy XET_NGHIEM_KHUYEN_NGHI và THUOC_KHUYEN_NGHI từ file CSV hoặc cấu trúc dữ liệu khác nếu muốn)

    print(f"\nLưu ý: Đây chỉ là chẩn đoán sơ bộ dựa trên mô hình AI. Bạn nên tham khảo ý kiến của bác sĩ để có chẩn đoán chính xác và kế hoạch điều trị phù hợp.")


if __name__ == "__main__":
    # Tải dữ liệu
    (X_train, y_train, TEN_BENH_LIST, MA_TRIEU_CHUNG_CHINH_LIST, TEN_TRIEU_CHUNG_CHINH_LIST,
     symptoms_df, tu_khoa_map, cau_hoi_lam_ro_map, ma_to_cauhoi_map, ma_to_ten_map) = load_data()

    # Xây dựng và huấn luyện mô hình
    num_features = X_train.shape[1]
    num_classes = len(TEN_BENH_LIST)
    model = build_and_train_model(X_train, y_train, num_features, num_classes)

    # Chạy chatbot
    if model:
        run_intelligent_chatbot(model, TEN_BENH_LIST, MA_TRIEU_CHUNG_CHINH_LIST, TEN_TRIEU_CHUNG_CHINH_LIST,
                                symptoms_df, tu_khoa_map, cau_hoi_lam_ro_map, ma_to_cauhoi_map, ma_to_ten_map)
    else:
        print("Kết thúc chương trình do lỗi huấn luyện mô hình.")

Huấn luyện mô hình với 53 mẫu, 34 đặc trưng, và 18 lớp bệnh.

Xin chào! Tôi là robot trợ lý sức khỏe ảo. Hãy mô tả các triệu chứng chính bạn đang gặp phải.

[Bot]: Dựa trên mô tả của bạn, tôi ghi nhận các triệu chứng sau:
- Sốt
- Đau đầu

[Bot]: Để hiểu rõ hơn, tôi xin hỏi thêm một số câu:

[Bot]: Cảm ơn bạn đã cung cấp thông tin. Đang phân tích...

--- Chẩn đoán với Xác suất và Độ không chắc chắn ---
Cúm: Xác suất = 0.067, Độ không chắc chắn (StdDev) = 0.030
Cảm lạnh: Xác suất = 0.096, Độ không chắc chắn (StdDev) = 0.076
COVID-19: Xác suất = 0.060, Độ không chắc chắn (StdDev) = 0.036
Dị ứng: Xác suất = 0.059, Độ không chắc chắn (StdDev) = 0.042
Viêm xoang: Xác suất = 0.084, Độ không chắc chắn (StdDev) = 0.066
Đau nửa đầu Migraine: Xác suất = 0.055, Độ không chắc chắn (StdDev) = 0.040
Căng thẳng Stress: Xác suất = 0.034, Độ không chắc chắn (StdDev) = 0.029
Viêm họng: Xác suất = 0.055, Độ không chắc chắn (StdDev) = 0.028
Trào ngược dạ dày thực quản: Xác suất = 0.052, Độ không chắc chắn 

: 